<a href="https://colab.research.google.com/github/ttycp3/Classdesign/blob/main/NLEQuestion3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#G5119: Natural Language Engineering

##Computer Based Examination, 2023

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [8]:
# update your candidate number here
candidate_number = 625937



#Question 3 (50 Marks)

This question is about Named Entity Recognition

In [18]:
### do not change the code in this cell
# make sure you run this cell
import nltk

# This is a list of sentences with NER tags which we will use in this question.
tagged_sents=[["tim_PER", "cook_PER", "is", "the", "current", "ceo", "of", "apple_ORG", "inc._ORG", ",", "headquartered", "in", "cupertino_LOC", "."],
              ["fiona_PER", "apple_PER", "was", "born", "in", "new_LOC", "york_LOC", "and", "her", "debut", "album", "was", "released", "by", "columbia_ORG", "records_ORG", "."],
              ["the_LOC", "big_LOC", "apple_LOC", "is", "a", "nickname", "for", "new_LOC", "york_LOC", "city_LOC", "."]]


a) Follow the steps below to preprocess the NER tagged sentences, producing a list of lists of tokens and a list of lists of NER tags.

i) Use the `.split()` method to separate the tokens from the NER tags in the list `tagged_sents`, and strip off the tags to produce a list of lists of strings, called `tokens`, corresponding to just the tokens in each sentence.

So, for example `[["Alice_PER", "runs"], ["Bob_PER", "walks"]]` would produce `[["Alice", "runs"], ["Bob", "walks"]]`.

(8 marks)

In [19]:

tokens = []
for sent in tagged_sents:
  token = []
  for word in sent:
    token.append(word.split('_')[0])
  tokens.append(token)


ii) Use the `.split()` method to separate the tokens from the NER tags in the list `tagged_sents`, producing a list of lists of NER tags, called `tags`. Tokens without an NER tag should be tagged with the letter `"O"`.

So, for example `[["Alice_PER", "runs"], ["chase", "Bob_PER"]]` would produce `[["PER", "O"], ["O", "PER"]]`.

(6 marks)

In [20]:
tags = []
for sent in tagged_sents:
  tag = []
  for word in sent:
    tag.append(word.split('_')[1] if len(word.split('_')) > 1 else 'O')
  tags.append(tag)


b) Now, follow the steps below to derive the transition and emission probabilities of a Hidden Markov Model for the NER tag sequences.

i) Describe the two assumptions that a Hidden Markov Model for sequence tagging is based on.

(4 marks)

1.Markov Property: The model assumes that the probability of a current state depends only on the previous state and not on any further past states, which is called the first-order Markov assumption. In other words, each element in the sequence is dependent solely on the immediately preceding element. Mathematically, this can be expressed as P(state(t) | state(1), ..., state(t-1)) = P(state(t) | state(t-1)), where 'state(t)' refers to the state at time 't'.

2.Observation Independence: HMMs also assume that the observation (or output) at a given time step is conditionally independent of all other observations given the underlying hidden state at that time. This means that once you know the hidden state, the presence or absence of another observation does not affect the likelihood of the current observation. Formally, this can be written as P(observation(t) | observation(1), ..., observation(t-1), state(1), ..., state(t)) = P(observation(t) | state(t)).

ii) Create a dictionary, called `nercounts`, containing the total counts for each tag in the list `tags`. The keys of `nercounts` should be tags and the values should be the frequencies of those tags.

(4 marks)

In [21]:
nercounts = dict()
for sent in tags:
  for tag in sent:
    if tag not in nercounts:
      nercounts[tag] = 0
    nercounts[tag] += 1


iii) Using the lists `tags` and `tokens`, calculate the emission probabilities: $$p(token|tag)$$

Put these probabilities in a dictionary of dictionaries, called `emission`, with outer keys being NER tags, inner keys being tokens and inner values being the emission probabilities.

(8 marks)

In [22]:
emission = dict()
for sent, tag in zip(tokens, tags):
  for token, tag in zip(sent, tag):
    if tag not in emission:
      emission[tag] = dict()
    if token not in emission[tag]:
      emission[tag][token] = 0
    emission[tag][token] += 1


iv) Using the list `tags`, calculate the transition probabilities: $$p(tag_i|tag_{i-1})$$

You will need to keep track of both the current tag, $tag_i$, and the previous tag, $tag_{i-1}$, and you will need to introduce a special tag, e.g. "START", for the previous tag at the beginning of a sequence.

For example, the sequence of tags `["PER", "O"]` would give rise to the following values of $i$, $tag_{i-1}$ and $tag_i$:

|i|$tag_{i-1}$|$tag_i$|
|---|---|---|
|0|START|PER|
|1|PER|O|

Calculate the conditional probabilities of $tag_i$ given $tag_{i-1}$ and put the results in a dictionary of dictionaries, called `transition`, with outer keys being previous tags, inner keys being current tags and inner values being transition probabilities.

(10 marks)

In [24]:
transition = dict()
start_tag = 'START'
for sent in tags:
  sent = [start_tag] + sent
  for i in range(len(sent)-1):
    prev_tag = sent[i]
    curr_tag = sent[i+1]
    if prev_tag not in transition:
      transition[prev_tag] = dict()
    if curr_tag not in transition[prev_tag]:
      transition[prev_tag][curr_tag] = 0
    transition[prev_tag][curr_tag] += 1

v) Given a sequence of tokens, we want to find the most probable sequence of tags corresponding to those tokens. The brute force approach would simply evaluate the probability of every sequence of tags. Explain why we would want to avoid that approach and describe an alternative algorithm.

(6 marks)

1.Exponential Complexity: The number of possible tag sequences grows exponentially with the length of the input token sequence. For example, if there are n different tags and a sentence has m words, then there would be n^m possible tag combinations. This quickly becomes unmanageable even for relatively short sentences.

2.Resource Intensive: Computing the probability for each sequence requires multiple multiplications, which is time-consuming and memory-intensive, especially when dealing with large vocabularies or complex models.

3.Practical Limitations: Even with substantial computational resources, the time required to compute probabilities for all possible sequences could exceed any reasonable processing time limit.

An alternative algorithm that avoids these issues is the Viterbi Algorithm. It is specifically designed to solve this problem efficiently by using dynamic programming. The Viterbi algorithm works as follows:

1.It maintains a probability matrix where each cell (i, j) represents the maximum probability of observing the first i tokens with the last tag being j.

2.At each step, it updates the probability matrix by considering the transition probabilities from previous tags to the current tag, along with the emission probabilities (the probability of the token given the tag).

3.After traversing the entire input sequence, it backtracks through the probability matrix to find the single path with the highest overall probability, which corresponds to the most probable sequence of tags.

vi) If we are tagging a sequence and the current token is `"apple"` and the previous tag is `"LOC"`, then we want know to which tag in the current position  would maximise the probability of seeing that token.

Define a function that takes a token, a previous tag, a dictionary of emission probabilities and a dictionary of transition probabilities and returns a dictionary of the probabilities $p(token, tag_i | tag_{i-1})$ for all values of $tag_i$.

Apply this function to the case where the current token is `"apple"` and the previous tag is `"LOC"`.

(4 marks)

In [32]:
def get_probabilities(token, prev_tag, emission, transition):
  probabilities = {}
  for tag in emission:
    if prev_tag in transition and tag in transition[prev_tag]:
      transition_prob = transition[prev_tag][tag]
    else:
      transition_prob = 0
    if tag in emission and token in emission[tag]:
      emission_prob = emission[tag][token]
    else:
      emission_prob = 0
    probabilities[tag] = transition_prob * emission_prob
  return probabilities
probabilities = get_probabilities("apple", "LOC", emission, transition)
print(probabilities)

{'PER': 0, 'O': 0, 'ORG': 0, 'LOC': 5}
